# 运筹优化求解器ortools教程：线性规划 (1)



## ❓ "什么是ORtools"

OR-Tools是一个谷歌开源的优化软件，可用于解决车辆路径规划、大规模调度、装箱问题、线性规划、整数规划、混合整数规划、图与网络以及其他复杂的运筹优化问题。

- 参考🔗：搜索引擎搜ortools，developers.google 开头的网站就是官网。官网有非常详细的入门教程。本教程即改编于它。
- 使用🔧：你可以通过Python / Java / C++ / C# 进行调用使用。本教程使用Python语言进行讲解。

在Python下安装ortools：进入你的Python环境，输入命令：

```text
python -m pip install --upgrade --user ortools
```

即可使用。




## 使用ortools进行LP求解
    
LP问题的定义和具体形式这里不再赘述，可以参考我的小红书前几个笔记，这里总结以下ortools求解LP的框架和思路：

1. 导入LP求解器（ortools把它包装在MPSolver接口中）；
2. 声明LP求解器，
3. 定义决策变量，
4. 定义约束条件，
5. 定义目标函数，
6. 调用LP求解器进行求解，显示解决方案

e.g. 我们以如下形式的LP为例进行演示。

$$\max \hspace{5pt} 3x + 4y$$

$$\begin{align}\begin{equation*}
\begin{cases}
x + 2y \leq 14 \\
3x - y \leq 0 \\
x - y \leq 2 \\
\end{cases}
\end{equation*}\end{align}$$

---------


## Stigler Diet Problem

现在，我们引入一个更加规模更加庞大的复杂的线性规划问题：斯蒂格勒饮食问题（The Stigler Diet Problem），该问题以诺贝尔经济学奖获得者乔治·斯蒂格勒的名字命名，问题描述如下：

人们所需的9种营养物质需要满足**推荐摄入限度**，给定9种物质的摄入限度，以及很多种不同食物的营养物质含量、这些食物的价格（已经经过标准化），问，能否从这些食物中找到食物摄入组合，以及组合中不同食物的购买量，==使得既能够满足九种营养物质的摄入限度，又能使花费最少呢==？


| 营养物质  |  推荐摄入限度   |         营养物质          | 推荐摄入限度 |
| :-------: | :-------------: | :-----------------------: | :----------: |
| Calories  | 3,000  calories |   Thiamine (Vitamin B1)   |    1.8 mg    |
|  Protein  |    70  grams    |  Riboflavin (Vitamin B2)  |    2.7 mg    |
|  Calcium  |    .8  grams    |          Niacin           |    18 mg     |
|   Iron    |     12  mg      | Ascorbic Acid (Vitamin C) |    75 mg     |
| Vitamin A |    5,000  IU    |            ---            |     ----     |

食品种类异常繁多，数据（部分）基本如下：

|      商品      |  单个  | 1939年价格（分） | 热量（千卡） | 蛋白质（克） | 钙（克） | 铁（毫克） | 维生素A（KIU） | 硫胺素（毫克） | 核黄素（毫克） | 烟酸（毫克） | 抗坏血酸（毫克） |
| :------------: | :----: | :--------------: | :----------: | :----------: | :------: | :--------: | :------------: | :------------: | :------------: | :----------: | :--------------: |
| 小麦粉（强化） |  10磅  |        36        |     44.7     |    14.11     |    2     |    365     |       0        |      55.4      |      33.3      |     441      |        0         |
|     通心粉     |  1磅   |       14.1       |     11.6     |     41.8     |   0.7    |     54     |       0        |      3.2       |      1.9       |      68      |        0         |
| 小麦片（强化） | 28盎司 |       24.2       |     11.8     |     37.7     |   14.4   |    175     |       0        |      14.4      |      8.8       |     114      |        0         |
|     玉米片     | 8盎司  |       7.1        |     11.4     |     25.2     |   0.1    |     56     |       0        |      13.5      |      2.3       |      68      |        0         |
|     玉米粉     |  1磅   |       4.6        |     36.0     |     89.7     |   1.7    |     99     |      30.9      |      17.4      |      7.9       |     106      |        0         |
|    粗玉米粉    | 24盎司 |       8.5        |     28.6     |     68.0     |   0.8    |     80     |       0        |      10.6      |      1.6       |     110      |        0         |
|      大米      |  1磅   |       7.5        |     21.2     |     46.0     |   0.6    |     41     |       0        |       2        |      4.8       |      60      |        0         |
|      元宵      |  1磅   |       7.1        |     25.3     |     90.7     |   5.1    |    341     |       0        |      37.1      |      8.9       |      64      |        0         |
| 白面包（强化） |  1磅   |       7.9        |     15.0     |     48.8     |   2.5    |    115     |       0        |      13.8      |      8.5       |     126      |        0         |
|    全麦面包    |  1磅   |       9.1        |     12.2     |     48.4     |   2.7    |    125     |       0        |      13.9      |      6.4       |     160      |        0         |
|     红面包     |  1磅   |       9.1        |  12.4	43.9   |     1.1      |   82	0   |    9.9     |       3        |       66       |       0        |
|     油蛋糕     |  1磅   |       24.8       |     8.0      |     13.0     |   0.4    |     31     |      18.9      |      2.8       |       3        |      17      |

如何求解这个问题呢？








In [9]:
from ortools.linear_solver import pywraplp

def LinearProgrammingExample():
    """Linear programming sample."""
    # 实例化一个Glop求解器
    solver = pywraplp.Solver.CreateSolver("GLOP")
    if not solver:
        return
    
    # 声明两个变量，让他们取值范围在0到正无穷
    x = [solver.NumVar(0, solver.infinity(), f"x{i}") for i in range(5)]
    # x = [solver.NumVar(0, solver.infinity(), f"x{i}") for i in range(2)]
    
    print("Number of variables =",\
        solver.NumVariables())

    A = [[2, -1, 1, 6, -5], [1,1,2,1,2]]
    # A = [[2,1],[-1,1],[1,2],[6,1],[-5,2]]
    b = [6, 3]
    # b = [3,4,6,7,1]
    c = [3, 4, 6, 7, 1]
    # c = [6,3]
    NUM_C = len(b)
    for k in range(NUM_C):
        solver.Add(sum([i * j for (i, j) in zip(A[k], x)]) >= b[k])
        # print(sum([i * j for (i, j) in zip(A[k], x)]))
        # print(b[k])
    
    # 约束 0: x + 2y <= 14.
    # solver.Add(-2 * y1 + 7 * y2 >= 8)
    # solver.Add(2 * y1 +  y2 >= 10)
    # solver.Add(2 * y1 +  5 * y2 >= 20)
    # solver.Add(- y1 +  3 * y2 >= 2)
    
    # solver.Add(2 * y1 + 1 * y2 >= 3)
    # solver.Add(-1 * y1 +  y2 >= 4)
    # solver.Add( y1 +  2 * y2 >= 6)
    # solver.Add(6 * y1 +  1 * y2 >= 7)
    # solver.Add(-5 * y1 +  2 * y2 >= 1)

    # 约束 1: 3x - y >= 0.
    # 约束 2: x - y <= 2.

    print("Number of constraints =", \
        solver.NumConstraints())

    # 目标函数: 最大化 3x + 4y.
    # solver.Maximize(8 * y1 + 11 * y2)
    solver.Maximize(sum([i * j for (i, j) in zip(c, x)]))
    print(sum([i * j for (i, j) in zip(c, x)]))

    # 求解！
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("Solution:")
        print("Objective value =", solver.Objective().Value())
        for i in range(len(x)):
            print(f"x{i+1} = ",x[i].solution_value())
        
    else:
        if status == pywraplp.Solver.INFEASIBLE:
            print("INFEASIBLE!")
        elif status == pywraplp.Solver.UNBOUNDED:
            print("UNBOUNDED!")
        # print(status)
        # print("The problem does not have an optimal solution.")

    print("\nAdvanced usage:")
    print("Problem solved in %f milliseconds" % solver.wall_time())
    print("Problem solved in %d iterations" % solver.iterations())
LinearProgrammingExample()

Number of variables = 5
Number of constraints = 2
((((((3 * x0) + 0) + (4 * x1)) + (6 * x2)) + (7 * x3)) + (1 * x4))
INFEASIBLE!

Advanced usage:
Problem solved in 0.000000 milliseconds
Problem solved in 0 iterations


### 应用：Stigler Diet Problem（斯蒂格勒饮食问题）

In [4]:
from ortools.linear_solver import pywraplp

def main():
    """Entry point of the program."""
    # 实例化具体问题
    # 不同营养需要满足的最小值
    nutrients = [
        ["Calories (kcal)", 3],
        ["Protein (g)", 70],
        ["Calcium (g)", 0.8],
        ["Iron (mg)", 12],
        ["Vitamin A (KIU)", 5],
        ["Vitamin B1 (mg)", 1.8],
        ["Vitamin B2 (mg)", 2.7],
        ["Niacin (mg)", 18],
        ["Vitamin C (mg)", 75],
    ]

    # 类型, 单位, 1939 年价格(cent), 卡路里(kcal), 蛋白质 (g),
    # 钙 (g), 铁 (mg), 维生素 A (KIU), 维生素 B1 (mg), 维生素 B2 (mg),
    # 镍 (mg), 维生素 C (mg)
    data = [
        # fmt: off
      ['Wheat Flour (Enriched)', '10 lb.', 36, 44.7, 1411, 2, 365, 0, 55.4, 33.3, 441, 0],
      ['Macaroni', '1 lb.', 14.1, 11.6, 418, 0.7, 54, 0, 3.2, 1.9, 68, 0],
      ['Wheat Cereal (Enriched)', '28 oz.', 24.2, 11.8, 377, 14.4, 175, 0, 14.4, 8.8, 114, 0],
      ['Corn Flakes', '8 oz.', 7.1, 11.4, 252, 0.1, 56, 0, 13.5, 2.3, 68, 0],
      ['Corn Meal', '1 lb.', 4.6, 36.0, 897, 1.7, 99, 30.9, 17.4, 7.9, 106, 0],
      ['Hominy Grits', '24 oz.', 8.5, 28.6, 680, 0.8, 80, 0, 10.6, 1.6, 110, 0],
      ['Rice', '1 lb.', 7.5, 21.2, 460, 0.6, 41, 0, 2, 4.8, 60, 0],
      ['Rolled Oats', '1 lb.', 7.1, 25.3, 907, 5.1, 341, 0, 37.1, 8.9, 64, 0],
      ['White Bread (Enriched)', '1 lb.', 7.9, 15.0, 488, 2.5, 115, 0, 13.8, 8.5, 126, 0],
      ['Whole Wheat Bread', '1 lb.', 9.1, 12.2, 484, 2.7, 125, 0, 13.9, 6.4, 160, 0],
      ['Rye Bread', '1 lb.', 9.1, 12.4, 439, 1.1, 82, 0, 9.9, 3, 66, 0],
      ['Pound Cake', '1 lb.', 24.8, 8.0, 130, 0.4, 31, 18.9, 2.8, 3, 17, 0],
      ['Soda Crackers', '1 lb.', 15.1, 12.5, 288, 0.5, 50, 0, 0, 0, 0, 0],
      ['Milk', '1 qt.', 11, 6.1, 310, 10.5, 18, 16.8, 4, 16, 7, 177],
      ['Evaporated Milk (can)', '14.5 oz.', 6.7, 8.4, 422, 15.1, 9, 26, 3, 23.5, 11, 60],
      ['Butter', '1 lb.', 30.8, 10.8, 9, 0.2, 3, 44.2, 0, 0.2, 2, 0],
      ['Oleomargarine', '1 lb.', 16.1, 20.6, 17, 0.6, 6, 55.8, 0.2, 0, 0, 0],
      ['Eggs', '1 doz.', 32.6, 2.9, 238, 1.0, 52, 18.6, 2.8, 6.5, 1, 0],
      ['Cheese (Cheddar)', '1 lb.', 24.2, 7.4, 448, 16.4, 19, 28.1, 0.8, 10.3, 4, 0],
      ['Cream', '1/2 pt.', 14.1, 3.5, 49, 1.7, 3, 16.9, 0.6, 2.5, 0, 17],
      ['Peanut Butter', '1 lb.', 17.9, 15.7, 661, 1.0, 48, 0, 9.6, 8.1, 471, 0],
      ['Mayonnaise', '1/2 pt.', 16.7, 8.6, 18, 0.2, 8, 2.7, 0.4, 0.5, 0, 0],
      ['Crisco', '1 lb.', 20.3, 20.1, 0, 0, 0, 0, 0, 0, 0, 0],
      ['Lard', '1 lb.', 9.8, 41.7, 0, 0, 0, 0.2, 0, 0.5, 5, 0],
      ['Sirloin Steak', '1 lb.', 39.6, 2.9, 166, 0.1, 34, 0.2, 2.1, 2.9, 69, 0],
      ['Round Steak', '1 lb.', 36.4, 2.2, 214, 0.1, 32, 0.4, 2.5, 2.4, 87, 0],
      ['Rib Roast', '1 lb.', 29.2, 3.4, 213, 0.1, 33, 0, 0, 2, 0, 0],
      ['Chuck Roast', '1 lb.', 22.6, 3.6, 309, 0.2, 46, 0.4, 1, 4, 120, 0],
      ['Plate', '1 lb.', 14.6, 8.5, 404, 0.2, 62, 0, 0.9, 0, 0, 0],
      ['Liver (Beef)', '1 lb.', 26.8, 2.2, 333, 0.2, 139, 169.2, 6.4, 50.8, 316, 525],
      ['Leg of Lamb', '1 lb.', 27.6, 3.1, 245, 0.1, 20, 0, 2.8, 3.9, 86, 0],
      ['Lamb Chops (Rib)', '1 lb.', 36.6, 3.3, 140, 0.1, 15, 0, 1.7, 2.7, 54, 0],
      ['Pork Chops', '1 lb.', 30.7, 3.5, 196, 0.2, 30, 0, 17.4, 2.7, 60, 0],
      ['Pork Loin Roast', '1 lb.', 24.2, 4.4, 249, 0.3, 37, 0, 18.2, 3.6, 79, 0],
      ['Bacon', '1 lb.', 25.6, 10.4, 152, 0.2, 23, 0, 1.8, 1.8, 71, 0],
      ['Ham, smoked', '1 lb.', 27.4, 6.7, 212, 0.2, 31, 0, 9.9, 3.3, 50, 0],
      ['Salt Pork', '1 lb.', 16, 18.8, 164, 0.1, 26, 0, 1.4, 1.8, 0, 0],
      ['Roasting Chicken', '1 lb.', 30.3, 1.8, 184, 0.1, 30, 0.1, 0.9, 1.8, 68, 46],
      ['Veal Cutlets', '1 lb.', 42.3, 1.7, 156, 0.1, 24, 0, 1.4, 2.4, 57, 0],
      ['Salmon, Pink (can)', '16 oz.', 13, 5.8, 705, 6.8, 45, 3.5, 1, 4.9, 209, 0],
      ['Apples', '1 lb.', 4.4, 5.8, 27, 0.5, 36, 7.3, 3.6, 2.7, 5, 544],
      ['Bananas', '1 lb.', 6.1, 4.9, 60, 0.4, 30, 17.4, 2.5, 3.5, 28, 498],
      ['Lemons', '1 doz.', 26, 1.0, 21, 0.5, 14, 0, 0.5, 0, 4, 952],
      ['Oranges', '1 doz.', 30.9, 2.2, 40, 1.1, 18, 11.1, 3.6, 1.3, 10, 1998],
      ['Green Beans', '1 lb.', 7.1, 2.4, 138, 3.7, 80, 69, 4.3, 5.8, 37, 862],
      ['Cabbage', '1 lb.', 3.7, 2.6, 125, 4.0, 36, 7.2, 9, 4.5, 26, 5369],
      ['Carrots', '1 bunch', 4.7, 2.7, 73, 2.8, 43, 188.5, 6.1, 4.3, 89, 608],
      ['Celery', '1 stalk', 7.3, 0.9, 51, 3.0, 23, 0.9, 1.4, 1.4, 9, 313],
      ['Lettuce', '1 head', 8.2, 0.4, 27, 1.1, 22, 112.4, 1.8, 3.4, 11, 449],
      ['Onions', '1 lb.', 3.6, 5.8, 166, 3.8, 59, 16.6, 4.7, 5.9, 21, 1184],
      ['Potatoes', '15 lb.', 34, 14.3, 336, 1.8, 118, 6.7, 29.4, 7.1, 198, 2522],
      ['Spinach', '1 lb.', 8.1, 1.1, 106, 0, 138, 918.4, 5.7, 13.8, 33, 2755],
      ['Sweet Potatoes', '1 lb.', 5.1, 9.6, 138, 2.7, 54, 290.7, 8.4, 5.4, 83, 1912],
      ['Peaches (can)', 'No. 2 1/2', 16.8, 3.7, 20, 0.4, 10, 21.5, 0.5, 1, 31, 196],
      ['Pears (can)', 'No. 2 1/2', 20.4, 3.0, 8, 0.3, 8, 0.8, 0.8, 0.8, 5, 81],
      ['Pineapple (can)', 'No. 2 1/2', 21.3, 2.4, 16, 0.4, 8, 2, 2.8, 0.8, 7, 399],
      ['Asparagus (can)', 'No. 2', 27.7, 0.4, 33, 0.3, 12, 16.3, 1.4, 2.1, 17, 272],
      ['Green Beans (can)', 'No. 2', 10, 1.0, 54, 2, 65, 53.9, 1.6, 4.3, 32, 431],
      ['Pork and Beans (can)', '16 oz.', 7.1, 7.5, 364, 4, 134, 3.5, 8.3, 7.7, 56, 0],
      ['Corn (can)', 'No. 2', 10.4, 5.2, 136, 0.2, 16, 12, 1.6, 2.7, 42, 218],
      ['Peas (can)', 'No. 2', 13.8, 2.3, 136, 0.6, 45, 34.9, 4.9, 2.5, 37, 370],
      ['Tomatoes (can)', 'No. 2', 8.6, 1.3, 63, 0.7, 38, 53.2, 3.4, 2.5, 36, 1253],
      ['Tomato Soup (can)', '10 1/2 oz.', 7.6, 1.6, 71, 0.6, 43, 57.9, 3.5, 2.4, 67, 862],
      ['Peaches, Dried', '1 lb.', 15.7, 8.5, 87, 1.7, 173, 86.8, 1.2, 4.3, 55, 57],
      ['Prunes, Dried', '1 lb.', 9, 12.8, 99, 2.5, 154, 85.7, 3.9, 4.3, 65, 257],
      ['Raisins, Dried', '15 oz.', 9.4, 13.5, 104, 2.5, 136, 4.5, 6.3, 1.4, 24, 136],
      ['Peas, Dried', '1 lb.', 7.9, 20.0, 1367, 4.2, 345, 2.9, 28.7, 18.4, 162, 0],
      ['Lima Beans, Dried', '1 lb.', 8.9, 17.4, 1055, 3.7, 459, 5.1, 26.9, 38.2, 93, 0],
      ['Navy Beans, Dried', '1 lb.', 5.9, 26.9, 1691, 11.4, 792, 0, 38.4, 24.6, 217, 0],
      ['Coffee', '1 lb.', 22.4, 0, 0, 0, 0, 0, 4, 5.1, 50, 0],
      ['Tea', '1/4 lb.', 17.4, 0, 0, 0, 0, 0, 0, 2.3, 42, 0],
      ['Cocoa', '8 oz.', 8.6, 8.7, 237, 3, 72, 0, 2, 11.9, 40, 0],
      ['Chocolate', '8 oz.', 16.2, 8.0, 77, 1.3, 39, 0, 0.9, 3.4, 14, 0],
      ['Sugar', '10 lb.', 51.7, 34.9, 0, 0, 0, 0, 0, 0, 0, 0],
      ['Corn Syrup', '24 oz.', 13.7, 14.7, 0, 0.5, 74, 0, 0, 0, 5, 0],
      ['Molasses', '18 oz.', 13.6, 9.0, 0, 10.3, 244, 0, 1.9, 7.5, 146, 0],
      ['Strawberry Preserves', '1 lb.', 20.5, 6.4, 11, 0.4, 7, 0.2, 0.2, 0.4, 3, 0],
        # fmt: on
    ]

    # 声明一个 Glop求解器
    solver = pywraplp.Solver.CreateSolver("GLOP")
    if not solver:
        return

    # 声明一个列表来存储我们的变量
    foods = [solver.NumVar(0.0, solver.infinity(), item[0]) for item in data]

    print("Number of variables =", solver.NumVariables())

    # 定义约束条件，每个营养一个约束
    constraints = []
    for i, nutrient in enumerate(nutrients):
        constraints.append(solver.Constraint(nutrient[1], solver.infinity()))
        for j, item in enumerate(data):
            constraints[i].SetCoefficient(foods[j], item[i + 3])

    print("Number of constraints =", solver.NumConstraints())

    # 目标函数：最小化食品的价格（价格标准化后）
    objective = solver.Objective()
    for food in foods:
        objective.SetCoefficient(food, 1)
    objective.SetMinimization()

    status = solver.Solve()

    # 检查问题是否有最优解
    if status != solver.OPTIMAL:
        print("The problem does not have an optimal solution!")
        if status == solver.FEASIBLE:
            print("A potentially suboptimal solution was found.")
        else:
            print("The solver could not solve the problem.")
            exit(1)

    # 展示购买每种食物所需的数量（以美元形式）
    nutrients_result = [0] * len(nutrients)
    print("\nAnnual Foods:")
    for i, food in enumerate(foods):
        if food.solution_value() > 0.0:
            print("{}: ${}".format(data[i][0], 365.0 * food.solution_value()))
            for j, _ in enumerate(nutrients):
                nutrients_result[j] += data[i][j + 3] * food.solution_value()
    print("\nOptimal annual price: ${:.4f}".format(365.0 * objective.Value()))

    print("\nNutrients per day:")
    for i, nutrient in enumerate(nutrients):
        print(
            "{}: {:.2f} (min {})".format(nutrient[0], nutrients_result[i], nutrient[1])
        )

    print("\nAdvanced usage:")
    print("Problem solved in ", solver.wall_time(), " milliseconds")
    print("Problem solved in ", solver.iterations(), " iterations")


if __name__ == "__main__":
    main()

Number of variables = 77
Number of constraints = 9

Annual Foods:
Wheat Flour (Enriched): $10.774457511918223
Liver (Beef): $0.6907834111074193
Cabbage: $4.093268864842877
Spinach: $1.8277960703546996
Navy Beans, Dried: $22.275425687243036

Optimal annual price: $39.6617

Nutrients per day:
Calories (kcal): 3.00 (min 3)
Protein (g): 147.41 (min 70)
Calcium (g): 0.80 (min 0.8)
Iron (mg): 60.47 (min 12)
Vitamin A (KIU): 5.00 (min 5)
Vitamin B1 (mg): 4.12 (min 1.8)
Vitamin B2 (mg): 2.70 (min 2.7)
Niacin (mg): 27.32 (min 18)
Vitamin C (mg): 75.00 (min 75)

Advanced usage:
Problem solved in  2  milliseconds
Problem solved in  14  iterations


### 尝试：如果是非标准形式的LP求解

$$\max \hspace{5pt} z = x_1 + 2 x_2 + x_3$$

$$\begin{align}\begin{equation*}
\begin{cases}
x_1 + 4x_2 - 2x_3 \geq 120 \\
x_1 + x_2 + x_3 = 60 \\
x_1, x_2, x_3 \geq 0
\end{cases}
\end{equation*}\end{align}$$

如果化成大M法，还需要考虑人工变量的问题，此时不如直接对约束条件进行处理。

> SCIP求解器：待补充

In [11]:
from ortools.linear_solver import pywraplp

def main():
    """Entry point of the program."""
    # 实例化具体问题
    # 目标函数及其参数
    targets = [
        ["x1", 3],
        ["x2", 4],
        ["x3", 6],
        ["x4", 7],
        ["x5", 1],
    ] # TODO: 解题需要注意数据格式
    data = [

        ["Constraint1", 2,-1,1,6,-5],
        ["Constraint2", 1,1,2,1,2]
        # 约束条件
        # TODO: 解题需要注意数据格式
    ]
    

    # 声明一个 Glop求解器
    solver = pywraplp.Solver.CreateSolver("GLOP")
    if not solver:
        return

    # 声明一个列表来存储我们的变量
    variables = [solver.NumVar(0.0, solver.infinity(), item[0]) for item in targets] # TODO: 解题需注意决策变量的范围

    print("Number of variables =", solver.NumVariables())

    # 定义约束条件，每个营养一个约束
    constraints = []
    
    # 定义约束（如果是等式约束怎么写？上下界相等即可）
    
    
    # TODO: 解题需注意决策变量的范围
    constraints.append(solver.Constraint(6, solver.infinity()))
    for i in range(solver.NumVariables()):
        constraints[0].SetCoefficient(variables[i], data[0][i + 1]) # 给约束条件把参数配上去
    constraints.append(solver.Constraint(3, solver.infinity()))
    for i in range(solver.NumVariables()):
        constraints[1].SetCoefficient(variables[i], data[1][i + 1])

    print("Number of constraints =", solver.NumConstraints())


    objective = solver.Objective()
    for idx,variable in enumerate(variables):
        # variable: [ (下限，上限，名称) ]
        objective.SetCoefficient(variable, targets[idx][1])
    objective.SetMaximization() # 求最大化

    status = solver.Solve()

    # 检查问题是否有最优解
    if status != solver.OPTIMAL:
        print("The problem does not have an optimal solution!")
        if status == solver.FEASIBLE:
            print("A potentially suboptimal solution was found.")
        elif status == solver.UNBOUNDED:
            print("A UNBOUNDED suboptimal solution was found.")
        else:
            print(status)
            print("The solver could not solve the problem.")
            exit(1)

    print("\nNumber:")
    for i, food in enumerate(variables):
        print("{}: ${}".format(targets[i][0], food.solution_value()))
    
    print("\nResult: ${:.4f}".format(objective.Value()))
    print("\nAdvanced usage:")
    print("Problem solved in ", solver.wall_time(), " milliseconds")
    print("Problem solved in ", solver.iterations(), " iterations")


if __name__ == "__main__":
    main()

Number of variables = 5
Number of constraints = 2
The problem does not have an optimal solution!
2
The solver could not solve the problem.

Number:
x1: $0.0
x2: $0.0
x3: $0.0
x4: $0.0
x5: $0.0

Result: $0.0000

Advanced usage:
Problem solved in  0  milliseconds
Problem solved in  0  iterations


E0000 00:00:1730707327.976643 42672478 linear_solver.cc:2025] No solution exists. MPSolverInterface::result_status_ = MPSOLVER_INFEASIBLE
E0000 00:00:1730707327.976659 42672478 linear_solver.cc:2025] No solution exists. MPSolverInterface::result_status_ = MPSOLVER_INFEASIBLE
E0000 00:00:1730707327.976663 42672478 linear_solver.cc:2025] No solution exists. MPSolverInterface::result_status_ = MPSOLVER_INFEASIBLE
E0000 00:00:1730707327.976666 42672478 linear_solver.cc:2025] No solution exists. MPSolverInterface::result_status_ = MPSOLVER_INFEASIBLE
E0000 00:00:1730707327.976670 42672478 linear_solver.cc:2025] No solution exists. MPSolverInterface::result_status_ = MPSOLVER_INFEASIBLE
E0000 00:00:1730707327.976673 42672478 linear_solver.cc:2025] No solution exists. MPSolverInterface::result_status_ = MPSOLVER_INFEASIBLE


### 另一道例题

> 对应机械工业出版社南大《运筹学》P28 退化与循环的例子

In [20]:
from ortools.linear_solver import pywraplp

def main():
    """Entry point of the program."""
    # 实例化具体问题
    # 目标函数及其参数
    targets = [
        ["x1", -3/4],
        ["x2", 150],
        ["x3", -1/50],
        ["x4", 6],
        ["x5", 0],
        ["x6", 0],
        ["x7", 0],
    ] # TODO: 解题需要注意数据格式
    data = [

        ["Constraint1", 1/4, -60, -1/25, 9, 1, 0,0],
        ["Constraint2", 1/2, -90,  -1/50, 3, 0, 1, 0],
        ["Constraint3", 0, 0, 1, 0 ,0, 0, 1]
        # 约束条件
        # TODO: 解题需要注意数据格式
    ]
    

    # 声明一个 Glop求解器
    solver = pywraplp.Solver.CreateSolver("GLOP")
    if not solver:
        return

    # 声明一个列表来存储我们的变量
    variables = [solver.NumVar(0.0, solver.infinity(), item[0]) for item in targets] # TODO: 解题需注意决策变量的范围

    print("Number of variables =", solver.NumVariables())

    # 定义约束条件，每个营养一个约束
    constraints = []
    
    # 定义约束（如果是等式约束怎么写？上下界相等即可）
    
    
    # TODO: 解题需注意决策变量的范围
    constraints.append(solver.Constraint(0, 0))
    for i in range(solver.NumVariables()):
        constraints[0].SetCoefficient(variables[i], data[0][i + 1]) # 给约束条件把参数配上去
    constraints.append(solver.Constraint(0, 0))
    for i in range(solver.NumVariables()):
        constraints[1].SetCoefficient(variables[i], data[1][i + 1])
    constraints.append(solver.Constraint(1, 1))
    for i in range(solver.NumVariables()):
        constraints[2].SetCoefficient(variables[i], data[2][i + 1])


    print("Number of constraints =", solver.NumConstraints())


    objective = solver.Objective()
    for idx,variable in enumerate(variables):
        # variable: [ (下限，上限，名称) ]
        objective.SetCoefficient(variable, targets[idx][1])
    objective.SetMinimization() # 求最大化

    status = solver.Solve()

    # 检查问题是否有最优解
    if status != solver.OPTIMAL:
        print("The problem does not have an optimal solution!")
        if status == solver.FEASIBLE:
            print("A potentially suboptimal solution was found.")
        else:
            print("The solver could not solve the problem.")
            exit(1)

    print("\nNumber:")
    for i, food in enumerate(variables):
        print("{}: ${}".format(targets[i][0], food.solution_value()))
    
    print("\nResult: ${:.4f}".format(objective.Value()))
    print("\nAdvanced usage:")
    print("Problem solved in ", solver.wall_time(), " milliseconds")
    print("Problem solved in ", solver.iterations(), " iterations")


if __name__ == "__main__":
    main()

Number of variables = 7
Number of constraints = 3

Number:
x1: $0.04
x2: $0.0
x3: $1.0
x4: $0.0
x5: $0.03
x6: $0.0
x7: $0.0

Result: $-0.0500

Advanced usage:
Problem solved in  8  milliseconds
Problem solved in  1  iterations


### 运输问题的线性规划建模

> 题目来自《运筹学》机械工业出版社 P74教材例3.12

In [21]:
from ortools.linear_solver import pywraplp

def main():
    """Entry point of the program."""
    # 实例化具体问题
    # 目标函数及其参数
    
    
    costs = [150, 220, 160, 230, 0, 170, 260, 180, 270, 0, 160, 220, 175, 235, 0, 190, 250, 205, 265, 0, 170, 230, 190, 250, 0, 180, 260, 200, 280, 0 , 99999, 99999, 150, 220, 0, 99999, 99999, 170, 260, 0, 99999, 99999, 160, 220, 0, 99999, 99999, 190, 250, 0, 99999, 99999, 170, 230, 0, 99999, 99999, 180, 260, 0]
    supplies = [100, 50, 200, 50, 500, 100, 100, 50, 200, 0, 500, 100]
    demands = [200, 600, 400, 700, 50]
    targets = []
    
    for idx, cost in enumerate(costs):
        targets.append([f"x{ idx // 5}{ idx % 5}", cost])
    
    # TODO: 解题需要注意数据格式
    
    data = []
    # TODO: 解题需要注意数据格式
    for i in range(12):
        data.append([f"Constraints{i}", 1, 1, 1, 1, 1])
        
    # 约束条件
    

    # 声明一个 Glop求解器
    solver = pywraplp.Solver.CreateSolver("GLOP")
    if not solver:
        return

    # 声明一个列表来存储我们的变量
    variables = [solver.NumVar(0.0, solver.infinity(), item[0]) for item in targets] # TODO: 解题需注意决策变量的范围

    print("Number of variables =", solver.NumVariables())

    # 定义约束条件，每个营养一个约束
    constraints = []
    
    # 定义约束（如果是等式约束怎么写？上下界相等即可）
    
    
    # TODO: 解题需注意决策变量的范围
    for i in range(12):
        constraints.append(solver.Constraint(supplies[i], supplies[i]))
        for j in range(5):
            constraints[i].SetCoefficient(variables[i * 5 + j], data[i][j + 1])
    
    for i in range(5):
        constraints.append(solver.Constraint(demands[i], demands[i]))
        for j in range(12):
            constraints[i + 12].SetCoefficient(variables[i + j * 5], data[j][i + 1])

    print("Number of constraints =", solver.NumConstraints())


    objective = solver.Objective()
    for idx,variable in enumerate(variables):
        # variable: [ (下限，上限，名称) ]
        objective.SetCoefficient(variable, targets[idx][1])
    objective.SetMinimization() # 求最大/小化

    status = solver.Solve()

    # 检查问题是否有最优解
    if status != solver.OPTIMAL:
        print("The problem does not have an optimal solution!")
        if status == solver.FEASIBLE:
            print("A potentially suboptimal solution was found.")
        else:
            print("The solver could not solve the problem.")
            exit(1)

    print("\nNumber:")
    for i, food in enumerate(variables):
        print("{}: ${}".format(targets[i][0], food.solution_value()))
    
    print("\nResult: ${:.4f}".format(objective.Value()))
    print("\nAdvanced usage:")
    print("Problem solved in ", solver.wall_time(), " milliseconds")
    print("Problem solved in ", solver.iterations(), " iterations")


if __name__ == "__main__":
    main()

Number of variables = 60
Number of constraints = 17

Number:
x00: $0.0
x01: $0.0
x02: $100.0
x03: $0.0
x04: $0.0
x10: $0.0
x11: $0.0
x12: $50.0
x13: $0.0
x14: $0.0
x20: $0.0
x21: $200.0
x22: $0.0
x23: $0.0
x24: $0.0
x30: $0.0
x31: $0.0
x32: $0.0
x33: $0.0
x34: $50.0
x40: $100.0
x41: $400.0
x42: $0.0
x43: $0.0
x44: $0.0
x50: $100.0
x51: $0.0
x52: $0.0
x53: $0.0
x54: $0.0
x60: $0.0
x61: $0.0
x62: $100.0
x63: $0.0
x64: $0.0
x70: $0.0
x71: $0.0
x72: $50.0
x73: $0.0
x74: $0.0
x80: $0.0
x81: $0.0
x82: $0.0
x83: $200.0
x84: $0.0
x90: $0.0
x91: $0.0
x92: $0.0
x93: $0.0
x94: $0.0
x100: $0.0
x101: $0.0
x102: $0.0
x103: $500.0
x104: $0.0
x110: $0.0
x111: $0.0
x112: $100.0
x113: $0.0
x114: $0.0

Result: $396500.0000

Advanced usage:
Problem solved in  28  milliseconds
Problem solved in  18  iterations


In [3]:
from ortools.linear_solver import pywraplp


def LinearProgrammingExample():
    """Linear programming sample."""
    # Instantiate a Glop solver, naming it LinearExample.
    solver = pywraplp.Solver.CreateSolver("GLOP")
    if not solver:
        return

    # Create the two variables and let them take on any non-negative value.
    variables = [solver.NumVar(0.0, solver.infinity(), f"x_{i}") for i in range(11)]

   

    print("Number of variables =", solver.NumVariables())

    # Constraint 0: x + 2y <= 14.
    solver.Add( 2 * variables[0] + variables[1] + 3 * variables[3] + 2 * variables[4] + variables[5] + 4 * variables[6]
                + 3 * variables[7] + 2 * variables[8]  >= 140)
    solver.Add( 1 * variables[0] + variables[1] + 1 * variables[3] + 2 * variables[4] + variables[5] + variables[8] * 2 + variables[10] * 3 >= 60)
    # Constraint 1: 3x - y >= 0.
    solver.Add( variables[2] * 2 + variables[5] + variables[6] + 2 * variables[7] + variables[9] * 7 >= 20)

    # Constraint 2: x - y <= 2.
    # solver.Add(x - z <= 2.0)

    print("Number of constraints =", solver.NumConstraints())

    # Objective function: 3x + 4y.
    solver.Minimize(2 * (variables[0] + variables[1] + variables[2]) + 3 * (variables[3] + variables[4] + variables[5]) + 
                    4* (variables[6] + variables[7] + variables[8] +  variables[9] +  variables[10]))

    # Solve the system.
    print(f"Solving with {solver.SolverVersion()}")
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("Solution:")
        print(f"Objective value = {solver.Objective().Value():0.1f}")
        for i in range(11):
            print(f"x_{i} = {variables[i].solution_value():0.1f}")
        # print(f"y = {y.solution_value():0.1f}")
    else:
        print("The problem does not have an optimal solution.")

    print("\nAdvanced usage:")
    print(f"Problem solved in {solver.wall_time():d} milliseconds")
    print(f"Problem solved in {solver.iterations():d} iterations")


LinearProgrammingExample()

Number of variables = 11
Number of constraints = 3
Solving with Glop solver v9.7.2996
Solution:
Objective value = 148.6
x_0 = 60.0
x_1 = 0.0
x_2 = 0.0
x_3 = 0.0
x_4 = 0.0
x_5 = 0.0
x_6 = 5.0
x_7 = 0.0
x_8 = 0.0
x_9 = 2.1
x_10 = 0.0

Advanced usage:
Problem solved in 1 milliseconds
Problem solved in 4 iterations


# Gurobi Simple LP

In [1]:
#/usr/bin/env python3.11

# Copyright 2024, Gurobi Optimization, LLC

# Solve the classic diet model, showing how to add constraints
# to an existing model.

import gurobipy as gp
from gurobipy import GRB


# Nutrition guidelines, based on
# USDA Dietary Guidelines for Americans, 2005
# http://www.health.gov/DietaryGuidelines/dga2005/

categories, minNutrition, maxNutrition = gp.multidict(
    {
        "calories": [1800, 2200],
        "protein": [91, GRB.INFINITY],
        "fat": [0, 65],
        "sodium": [0, 1779],
    }
)

foods, cost = gp.multidict(
    {
        "hamburger": 2.49,
        "chicken": 2.89,
        "hot dog": 1.50,
        "fries": 1.89,
        "macaroni": 2.09,
        "pizza": 1.99,
        "salad": 2.49,
        "milk": 0.89,
        "ice cream": 1.59,
    }
)

# Nutrition values for the foods
nutritionValues = {
    ("hamburger", "calories"): 410,
    ("hamburger", "protein"): 24,
    ("hamburger", "fat"): 26,
    ("hamburger", "sodium"): 730,
    ("chicken", "calories"): 420,
    ("chicken", "protein"): 32,
    ("chicken", "fat"): 10,
    ("chicken", "sodium"): 1190,
    ("hot dog", "calories"): 560,
    ("hot dog", "protein"): 20,
    ("hot dog", "fat"): 32,
    ("hot dog", "sodium"): 1800,
    ("fries", "calories"): 380,
    ("fries", "protein"): 4,
    ("fries", "fat"): 19,
    ("fries", "sodium"): 270,
    ("macaroni", "calories"): 320,
    ("macaroni", "protein"): 12,
    ("macaroni", "fat"): 10,
    ("macaroni", "sodium"): 930,
    ("pizza", "calories"): 320,
    ("pizza", "protein"): 15,
    ("pizza", "fat"): 12,
    ("pizza", "sodium"): 820,
    ("salad", "calories"): 320,
    ("salad", "protein"): 31,
    ("salad", "fat"): 12,
    ("salad", "sodium"): 1230,
    ("milk", "calories"): 100,
    ("milk", "protein"): 8,
    ("milk", "fat"): 2.5,
    ("milk", "sodium"): 125,
    ("ice cream", "calories"): 330,
    ("ice cream", "protein"): 8,
    ("ice cream", "fat"): 10,
    ("ice cream", "sodium"): 180,
}

# Model
m = gp.Model("diet")

# Create decision variables for the foods to buy
buy = m.addVars(foods, name="buy")

# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
#
# buy = {}
# for f in foods:
#   buy[f] = m.addVar(name=f)

# The objective is to minimize the costs
m.setObjective(buy.prod(cost), GRB.MINIMIZE)

# Using looping constructs, the preceding statement would be:
#
# m.setObjective(sum(buy[f]*cost[f] for f in foods), GRB.MINIMIZE)

# Nutrition constraints
m.addConstrs(
    (
        gp.quicksum(nutritionValues[f, c] * buy[f] for f in foods)
        == [minNutrition[c], maxNutrition[c]]
        for c in categories
    ),
    "_",
)

# Using looping constructs, the preceding statement would be:
#
# for c in categories:
#  m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods),
#             minNutrition[c], maxNutrition[c], c)


def printSolution():
    if m.status == GRB.OPTIMAL:
        print(f"\nCost: {m.ObjVal:g}")
        print("\nBuy:")
        for f in foods:
            if buy[f].X > 0.0001:
                print(f"{f} {buy[f].X:g}")
    else:
        print("No solution")


# Solve
m.optimize()
printSolution()

print("\nAdding constraint: at most 6 servings of dairy")
m.addConstr(buy.sum(["milk", "ice cream"]) <= 6, "limit_dairy")

constrs = m.getConstrs()

# Solve
m.optimize()
printSolution()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 12 columns and 39 nonzeros
Model fingerprint: 0x33ddb849
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [6e+01, 2e+03]
  RHS range        [6e+01, 2e+03]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 4 rows, 10 columns, 37 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.472500e+02   0.000000e+00      0s
       4    1.1828861e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.182886111e+01

Cost: 11.8289

Buy:
hamburger 0.604514
milk 6.97014
ice cream 2.59132

Adding constraint: at most 6 serving

In [5]:
##@file tutorial/puzzle.py
# @brief solve a simple puzzle using SCIP
"""
On a beach there are octopuses, turtles and cranes.
The total number of legs of all animals is 80, while the number of heads is 32.
What are the minimum numbers of turtles and octopuses, respectively?

Copyright (c) by Joao Pedro PEDROSO and Mikio KUBO, 2012
"""
from pyscipopt import Model

model = Model("puzzle")
x = model.addVar(vtype="I", name="octopusses")
y = model.addVar(vtype="I", name="turtles")
z = model.addVar(vtype="I", name="cranes")

# Set up constraint for number of heads
model.addCons(x + y + z == 32, name="Heads")

# Set up constraint for number of legs
model.addCons(8 * x + 4 * y + 2 * z == 80, name="Legs")

# Set objective function
model.setObjective(x + y, "minimize")

model.hideOutput()
model.optimize()

# solution = model.getBestSol()

print("Optimal value:", model.getObjVal())
print((x.name, y.name, z.name), " = ", (model.getVal(x), model.getVal(y), model.getVal(z)))

Optimal value: 4.0
('octopusses', 'turtles', 'cranes')  =  (2.0, 2.0, 28.0)


In [15]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp

try:
    # Create a new model
    m = gp.Model("matrix1")

    # Create variables
    x = m.addMVar((5,5), vtype=GRB.CONTINUOUS, name="x")

    # Set objective
    obj = np.array([[-4,5,3,2,9999],[5,-1,2,9999,4],[3,2,-3,5,5],[2,9999,5,-3,6],[9999,4,5,6,-5]])
    
    # res = x @ obj
    # print(res)

    m.setObjective(800 + gp.quicksum(gp.quicksum(k for k in x * obj)), GRB.MINIMIZE)

    # Build (sparse) constraint matrix
    # val = np.array([1.0, 2.0, 3.0, -1.0, -1.0])
    # row = np.array([0, 0, 0, 1, 1])
    # col = np.array([0, 1, 2, 0, 1])

    # A = np.array([[2, -1, 1, 6, -5], [1,1,2,1,2]])

    # # Build rhs vector
    # rhs = np.array([6, 3])

    # Add constraints
    m.addConstr(x @ np.array([1,1,1,1,1]) == np.array([60,90,50,50,50]), name="S")
    m.addConstr(x.T @ np.array([1,1,1,1,1]) == np.array([50,50,50,80,70]), name="R")

    # Optimize model
    m.optimize()

    print(x.X)
    print(f"Obj: {m.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 10 rows, 25 columns and 50 nonzeros
Model fingerprint: 0x7731a60b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 9e+01]
Presolve time: 0.00s
Presolved: 10 rows, 25 columns, 50 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.6000000e+31   1.000000e+31   1.600000e+01      0s
       9    3.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.000000000e+02
[[50.  0.  0. 10.  0.]
 [ 0. 50. 20.  0. 20.]
 [ 0.  0. 30. 20.  0.]
 [ 0.  0.  0. 50.  0.]
 [ 0.  0.  0.  0. 50.]]
Obj: 300


In [12]:
a = np.array([1,2,3,4])
b = np.array([4,5,6,7])
print(a * b)

[ 4 10 18 28]


In [20]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp

try:
    # Create a new model
    m = gp.Model("matrix1")

    # Create variables
    x = m.addMVar(shape=4, lb = 0, vtype=GRB.CONTINUOUS, name="x")

    # Set objective
    obj = np.array([8, 10, 20, 2])
    m.setObjective(obj @ x, GRB.MAXIMIZE)

    # Build (sparse) constraint matrix
    # val = np.array([1.0, 2.0, 3.0, -1.0, -1.0])
    # row = np.array([0, 0, 0, 1, 1])
    # col = np.array([0, 1, 2, 0, 1])

    # A = np.array([[2,1],[-1,1],[1,2],[6,1],[-5,2]])
    A = np.array([[-2,2,2,-1], [7,1,5,3]])

    # Build rhs vector
    rhs = np.array([8, 11])

    # Add constraints
    m.addConstr(A @ x == rhs, name="c")

    # Optimize model
    m.optimize()

    print(x.X)
    print(f"Obj: {m.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0x555d5050
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 1e+01]
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2000000e+31   2.375000e+30   4.200000e+01      0s
       2    5.7500000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.750000000e+01
[0.   2.25 1.75 0.  ]
Obj: 57.5


/var/folders/wp/zfkgqzvj2lq6fk2gy0rbmq6m0000gn/T/ipykernel_21486/940427168.py:29: DeprecationWarning: `np.chararray` is deprecated and will be removed from the main namespace in the future. Use an array with a string or bytes dtype instead.
  m.addConstr(A @ x == rhs, name="c")


In [29]:
"""The Stigler diet problem.

A description of the problem can be found here:
https://en.wikipedia.org/wiki/Stigler_diet.
"""
from ortools.linear_solver import pywraplp


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # Nutrient minimums.
    nutrients = [
        ["Calories (kcal)", 3],
        ["Protein (g)", 70],
        ["Calcium (g)", 0.8],
        ["Iron (mg)", 12],
        ["Vitamin A (KIU)", 5],
        ["Vitamin B1 (mg)", 1.8],
        ["Vitamin B2 (mg)", 2.7],
        ["Niacin (mg)", 18],
        ["Vitamin C (mg)", 75],
    ]

    # Commodity, Unit, 1939 price (cents), Calories (kcal), Protein (g),
    # Calcium (g), Iron (mg), Vitamin A (KIU), Vitamin B1 (mg), Vitamin B2 (mg),
    # Niacin (mg), Vitamin C (mg)
    
    data = [
        # fmt: off
      ['Wheat Flour (Enriched)', '10 lb.', 36, 44.7, 1411, 2, 365, 0, 55.4, 33.3, 441, 0],
      ['Macaroni', '1 lb.', 14.1, 11.6, 418, 0.7, 54, 0, 3.2, 1.9, 68, 0],
      ['Wheat Cereal (Enriched)', '28 oz.', 24.2, 11.8, 377, 14.4, 175, 0, 14.4, 8.8, 114, 0],
      ['Corn Flakes', '8 oz.', 7.1, 11.4, 252, 0.1, 56, 0, 13.5, 2.3, 68, 0],
      ['Corn Meal', '1 lb.', 4.6, 36.0, 897, 1.7, 99, 30.9, 17.4, 7.9, 106, 0],
      ['Hominy Grits', '24 oz.', 8.5, 28.6, 680, 0.8, 80, 0, 10.6, 1.6, 110, 0],
      ['Rice', '1 lb.', 7.5, 21.2, 460, 0.6, 41, 0, 2, 4.8, 60, 0],
      ['Rolled Oats', '1 lb.', 7.1, 25.3, 907, 5.1, 341, 0, 37.1, 8.9, 64, 0],
      ['White Bread (Enriched)', '1 lb.', 7.9, 15.0, 488, 2.5, 115, 0, 13.8, 8.5, 126, 0],
      ['Whole Wheat Bread', '1 lb.', 9.1, 12.2, 484, 2.7, 125, 0, 13.9, 6.4, 160, 0],
      ['Rye Bread', '1 lb.', 9.1, 12.4, 439, 1.1, 82, 0, 9.9, 3, 66, 0],
      ['Pound Cake', '1 lb.', 24.8, 8.0, 130, 0.4, 31, 18.9, 2.8, 3, 17, 0],
      ['Soda Crackers', '1 lb.', 15.1, 12.5, 288, 0.5, 50, 0, 0, 0, 0, 0],
      ['Milk', '1 qt.', 11, 6.1, 310, 10.5, 18, 16.8, 4, 16, 7, 177],
      ['Evaporated Milk (can)', '14.5 oz.', 6.7, 8.4, 422, 15.1, 9, 26, 3, 23.5, 11, 60],
      ['Butter', '1 lb.', 30.8, 10.8, 9, 0.2, 3, 44.2, 0, 0.2, 2, 0],
      ['Oleomargarine', '1 lb.', 16.1, 20.6, 17, 0.6, 6, 55.8, 0.2, 0, 0, 0],
      ['Eggs', '1 doz.', 32.6, 2.9, 238, 1.0, 52, 18.6, 2.8, 6.5, 1, 0],
      ['Cheese (Cheddar)', '1 lb.', 24.2, 7.4, 448, 16.4, 19, 28.1, 0.8, 10.3, 4, 0],
      ['Cream', '1/2 pt.', 14.1, 3.5, 49, 1.7, 3, 16.9, 0.6, 2.5, 0, 17],
      ['Peanut Butter', '1 lb.', 17.9, 15.7, 661, 1.0, 48, 0, 9.6, 8.1, 471, 0],
      ['Mayonnaise', '1/2 pt.', 16.7, 8.6, 18, 0.2, 8, 2.7, 0.4, 0.5, 0, 0],
      ['Crisco', '1 lb.', 20.3, 20.1, 0, 0, 0, 0, 0, 0, 0, 0],
      ['Lard', '1 lb.', 9.8, 41.7, 0, 0, 0, 0.2, 0, 0.5, 5, 0],
      ['Sirloin Steak', '1 lb.', 39.6, 2.9, 166, 0.1, 34, 0.2, 2.1, 2.9, 69, 0],
      ['Round Steak', '1 lb.', 36.4, 2.2, 214, 0.1, 32, 0.4, 2.5, 2.4, 87, 0],
      ['Rib Roast', '1 lb.', 29.2, 3.4, 213, 0.1, 33, 0, 0, 2, 0, 0],
      ['Chuck Roast', '1 lb.', 22.6, 3.6, 309, 0.2, 46, 0.4, 1, 4, 120, 0],
      ['Plate', '1 lb.', 14.6, 8.5, 404, 0.2, 62, 0, 0.9, 0, 0, 0],
      ['Liver (Beef)', '1 lb.', 26.8, 2.2, 333, 0.2, 139, 169.2, 6.4, 50.8, 316, 525],
      ['Leg of Lamb', '1 lb.', 27.6, 3.1, 245, 0.1, 20, 0, 2.8, 3.9, 86, 0],
      ['Lamb Chops (Rib)', '1 lb.', 36.6, 3.3, 140, 0.1, 15, 0, 1.7, 2.7, 54, 0],
      ['Pork Chops', '1 lb.', 30.7, 3.5, 196, 0.2, 30, 0, 17.4, 2.7, 60, 0],
      ['Pork Loin Roast', '1 lb.', 24.2, 4.4, 249, 0.3, 37, 0, 18.2, 3.6, 79, 0],
      ['Bacon', '1 lb.', 25.6, 10.4, 152, 0.2, 23, 0, 1.8, 1.8, 71, 0],
      ['Ham, smoked', '1 lb.', 27.4, 6.7, 212, 0.2, 31, 0, 9.9, 3.3, 50, 0],
      ['Salt Pork', '1 lb.', 16, 18.8, 164, 0.1, 26, 0, 1.4, 1.8, 0, 0],
      ['Roasting Chicken', '1 lb.', 30.3, 1.8, 184, 0.1, 30, 0.1, 0.9, 1.8, 68, 46],
      ['Veal Cutlets', '1 lb.', 42.3, 1.7, 156, 0.1, 24, 0, 1.4, 2.4, 57, 0],
      ['Salmon, Pink (can)', '16 oz.', 13, 5.8, 705, 6.8, 45, 3.5, 1, 4.9, 209, 0],
      ['Apples', '1 lb.', 4.4, 5.8, 27, 0.5, 36, 7.3, 3.6, 2.7, 5, 544],
      ['Bananas', '1 lb.', 6.1, 4.9, 60, 0.4, 30, 17.4, 2.5, 3.5, 28, 498],
      ['Lemons', '1 doz.', 26, 1.0, 21, 0.5, 14, 0, 0.5, 0, 4, 952],
      ['Oranges', '1 doz.', 30.9, 2.2, 40, 1.1, 18, 11.1, 3.6, 1.3, 10, 1998],
      ['Green Beans', '1 lb.', 7.1, 2.4, 138, 3.7, 80, 69, 4.3, 5.8, 37, 862],
      ['Cabbage', '1 lb.', 3.7, 2.6, 125, 4.0, 36, 7.2, 9, 4.5, 26, 5369],
      ['Carrots', '1 bunch', 4.7, 2.7, 73, 2.8, 43, 188.5, 6.1, 4.3, 89, 608],
      ['Celery', '1 stalk', 7.3, 0.9, 51, 3.0, 23, 0.9, 1.4, 1.4, 9, 313],
      ['Lettuce', '1 head', 8.2, 0.4, 27, 1.1, 22, 112.4, 1.8, 3.4, 11, 449],
      ['Onions', '1 lb.', 3.6, 5.8, 166, 3.8, 59, 16.6, 4.7, 5.9, 21, 1184],
      ['Potatoes', '15 lb.', 34, 14.3, 336, 1.8, 118, 6.7, 29.4, 7.1, 198, 2522],
      ['Spinach', '1 lb.', 8.1, 1.1, 106, 0, 138, 918.4, 5.7, 13.8, 33, 2755],
      ['Sweet Potatoes', '1 lb.', 5.1, 9.6, 138, 2.7, 54, 290.7, 8.4, 5.4, 83, 1912],
      ['Peaches (can)', 'No. 2 1/2', 16.8, 3.7, 20, 0.4, 10, 21.5, 0.5, 1, 31, 196],
      ['Pears (can)', 'No. 2 1/2', 20.4, 3.0, 8, 0.3, 8, 0.8, 0.8, 0.8, 5, 81],
      ['Pineapple (can)', 'No. 2 1/2', 21.3, 2.4, 16, 0.4, 8, 2, 2.8, 0.8, 7, 399],
      ['Asparagus (can)', 'No. 2', 27.7, 0.4, 33, 0.3, 12, 16.3, 1.4, 2.1, 17, 272],
      ['Green Beans (can)', 'No. 2', 10, 1.0, 54, 2, 65, 53.9, 1.6, 4.3, 32, 431],
      ['Pork and Beans (can)', '16 oz.', 7.1, 7.5, 364, 4, 134, 3.5, 8.3, 7.7, 56, 0],
      ['Corn (can)', 'No. 2', 10.4, 5.2, 136, 0.2, 16, 12, 1.6, 2.7, 42, 218],
      ['Peas (can)', 'No. 2', 13.8, 2.3, 136, 0.6, 45, 34.9, 4.9, 2.5, 37, 370],
      ['Tomatoes (can)', 'No. 2', 8.6, 1.3, 63, 0.7, 38, 53.2, 3.4, 2.5, 36, 1253],
      ['Tomato Soup (can)', '10 1/2 oz.', 7.6, 1.6, 71, 0.6, 43, 57.9, 3.5, 2.4, 67, 862],
      ['Peaches, Dried', '1 lb.', 15.7, 8.5, 87, 1.7, 173, 86.8, 1.2, 4.3, 55, 57],
      ['Prunes, Dried', '1 lb.', 9, 12.8, 99, 2.5, 154, 85.7, 3.9, 4.3, 65, 257],
      ['Raisins, Dried', '15 oz.', 9.4, 13.5, 104, 2.5, 136, 4.5, 6.3, 1.4, 24, 136],
      ['Peas, Dried', '1 lb.', 7.9, 20.0, 1367, 4.2, 345, 2.9, 28.7, 18.4, 162, 0],
      ['Lima Beans, Dried', '1 lb.', 8.9, 17.4, 1055, 3.7, 459, 5.1, 26.9, 38.2, 93, 0],
      ['Navy Beans, Dried', '1 lb.', 5.9, 26.9, 1691, 11.4, 792, 0, 38.4, 24.6, 217, 0],
      ['Coffee', '1 lb.', 22.4, 0, 0, 0, 0, 0, 4, 5.1, 50, 0],
      ['Tea', '1/4 lb.', 17.4, 0, 0, 0, 0, 0, 0, 2.3, 42, 0],
      ['Cocoa', '8 oz.', 8.6, 8.7, 237, 3, 72, 0, 2, 11.9, 40, 0],
      ['Chocolate', '8 oz.', 16.2, 8.0, 77, 1.3, 39, 0, 0.9, 3.4, 14, 0],
      ['Sugar', '10 lb.', 51.7, 34.9, 0, 0, 0, 0, 0, 0, 0, 0],
      ['Corn Syrup', '24 oz.', 13.7, 14.7, 0, 0.5, 74, 0, 0, 0, 5, 0],
      ['Molasses', '18 oz.', 13.6, 9.0, 0, 10.3, 244, 0, 1.9, 7.5, 146, 0],
      ['Strawberry Preserves', '1 lb.', 20.5, 6.4, 11, 0.4, 7, 0.2, 0.2, 0.4, 3, 0],
        # fmt: on
    ]
    import numpy as np
    popl = []
    for d in data:
        popl.append(d[3:])
    g = np.array(popl).T
    for kd in g:
        print(list(map(float,list(kd))))

    # Instantiate a Glop solver and naming it.
    solver = pywraplp.Solver.CreateSolver("GLOP")
    if not solver:
        return

    # Declare an array to hold our variables.
    foods = [solver.NumVar(0.0, solver.infinity(), item[0]) for item in data]

    print("Number of variables =", solver.NumVariables())

    # Create the constraints, one per nutrient.
    constraints = []
    for i, nutrient in enumerate(nutrients):
        constraints.append(solver.Constraint(nutrient[1], solver.infinity()))
        for j, item in enumerate(data):
            constraints[i].SetCoefficient(foods[j], item[i + 3])

    print("Number of constraints =", solver.NumConstraints())

    # Objective function: Minimize the sum of (price-normalized) foods.
    objective = solver.Objective()
    for food in foods:
        objective.SetCoefficient(food, 1)
    objective.SetMinimization()

    print(f"Solving with {solver.SolverVersion()}")
    status = solver.Solve()

    # Check that the problem has an optimal solution.
    if status != solver.OPTIMAL:
        print("The problem does not have an optimal solution!")
        if status == solver.FEASIBLE:
            print("A potentially suboptimal solution was found.")
        else:
            print("The solver could not solve the problem.")
            exit(1)

    # Display the amounts (in dollars) to purchase of each food.
    nutrients_result = [0] * len(nutrients)
    print("\nAnnual Foods:")
    for i, food in enumerate(foods):
        if food.solution_value() > 0.0:
            print("{}: ${}".format(data[i][0], 365.0 * food.solution_value()))
            for j, _ in enumerate(nutrients):
                nutrients_result[j] += data[i][j + 3] * food.solution_value()
    print("\nOptimal annual price: ${:.4f}".format(365.0 * objective.Value()))

    print("\nNutrients per day:")
    for i, nutrient in enumerate(nutrients):
        print(
            "{}: {:.2f} (min {})".format(nutrient[0], nutrients_result[i], nutrient[1])
        )

    print("\nAdvanced usage:")
    print(f"Problem solved in {solver.wall_time():d} milliseconds")
    print(f"Problem solved in {solver.iterations():d} iterations")


if __name__ == "__main__":
    main()

[44.7, 11.6, 11.8, 11.4, 36.0, 28.6, 21.2, 25.3, 15.0, 12.2, 12.4, 8.0, 12.5, 6.1, 8.4, 10.8, 20.6, 2.9, 7.4, 3.5, 15.7, 8.6, 20.1, 41.7, 2.9, 2.2, 3.4, 3.6, 8.5, 2.2, 3.1, 3.3, 3.5, 4.4, 10.4, 6.7, 18.8, 1.8, 1.7, 5.8, 5.8, 4.9, 1.0, 2.2, 2.4, 2.6, 2.7, 0.9, 0.4, 5.8, 14.3, 1.1, 9.6, 3.7, 3.0, 2.4, 0.4, 1.0, 7.5, 5.2, 2.3, 1.3, 1.6, 8.5, 12.8, 13.5, 20.0, 17.4, 26.9, 0.0, 0.0, 8.7, 8.0, 34.9, 14.7, 9.0, 6.4]
[1411.0, 418.0, 377.0, 252.0, 897.0, 680.0, 460.0, 907.0, 488.0, 484.0, 439.0, 130.0, 288.0, 310.0, 422.0, 9.0, 17.0, 238.0, 448.0, 49.0, 661.0, 18.0, 0.0, 0.0, 166.0, 214.0, 213.0, 309.0, 404.0, 333.0, 245.0, 140.0, 196.0, 249.0, 152.0, 212.0, 164.0, 184.0, 156.0, 705.0, 27.0, 60.0, 21.0, 40.0, 138.0, 125.0, 73.0, 51.0, 27.0, 166.0, 336.0, 106.0, 138.0, 20.0, 8.0, 16.0, 33.0, 54.0, 364.0, 136.0, 136.0, 63.0, 71.0, 87.0, 99.0, 104.0, 1367.0, 1055.0, 1691.0, 0.0, 0.0, 237.0, 77.0, 0.0, 0.0, 0.0, 11.0]
[2.0, 0.7, 14.4, 0.1, 1.7, 0.8, 0.6, 5.1, 2.5, 2.7, 1.1, 0.4, 0.5, 10.5, 15.1, 0.

In [ ]:
import heapq

def topKFrequent(self, nums: List[int], k: int) -> List[int]:
    freqmap = dict()
    for num in nums:
        if num in freqmap:
            freqmap[num] += 1
        else:
            freqmap[num] = 1
    minHeap = []
    for num, freq in freqmap.items():
        if len(minHeap) < k:
            heapq.heappush(minHeap, (freq, num))
        elif freq > minHeap[0][0]:
            heapq.heappushpop(minHeap, (freq, num))
    
    topK = [num for freq, num in minHeap]
    return topK
